# Deploying App in Voila
> App was deployed on an Amazon AWS spot instance. Final app source code available at [https://github.com/timcu/fast-tabulous-app/blob/main/fast-tabulous-with-db.ipynb](https://github.com/timcu/fast-tabulous-app/blob/main/fast-tabulous-with-db.ipynb) . Final app can be used at [https://tabulous.pythonator.com](https://tabulous.pythonator.com)

- toc: true 
- badges: true
- comments: true
- categories: [Deployment, ipywidgets, TabularLearner, CPU, Sensitivity Analysis, Voila, Ubuntu, nginx]
- author: Tim Cummings

## Issues influencing deployment

- Early trials were showing memory problems in 16GB of RAM (app not loading properly) so 32GB RAM chosen
- Voila not showing ipywidgets properly when deployed through apache2 so nginx chosen for easier proxying of web sockets
- App was considered to be not mission critical so Amazon Spot Instance would be adequate giving 70% saving over Amazon On Demand Instance. 
- Deploy in Sydney data centre so most economic 32GB spot instance type available is t3.2xlarge (set max cost $0.20 per hour)
- Choose latest LTS Ubuntu 20.04

## Configure Ubuntu

After starting instance we get an IP address. We have a domain name for this instance `tabulous.pythonator.com` which we register with our DNS provider.

Now we can connect to the server using 

    ssh ubuntu@tabulous.pythonator.com
   
Install required packages

    sudo apt install python3.8-venv mc nginx python3-certbot-nginx mongodb

- Python `venv` to create python virtual environments
- Midnight Commander `mc` for ease of file operations
- Web server `nginx` for ease of web socket proxying
- Let's encrypt `certbot` to create https certificate
- Database `mongodb` to improve app load times rather than loading all data from csv to DataFrame every app load

## Configure Voila/Jupyter

Create a directory called jupyter from which will run jupyter and voila. Install a python virtual environment so not to pollute system python.

    mkdir -p jupyter/homesite-quote
    cd jupyter
    python3 -m venv venv
    source venv/bin/activate

### Configure python virtual environment for fastai, jupyter, ipywidgets, voila, kaggle and pymongo
    
    nano requirements.txt
        # jupyter/requirements.txt
        fastai==2.4.1
        fastbook==0.0.16
        fastcore==1.3.20
        fastprogress==1.0.0
        fastrelease==0.1.11
        ipykernel==5.5.5
        ipython==7.24.1
        ipython-genutils==0.2.0
        ipywidgets==7.6.3
        jupyter==1.0.0
        jupyter-client==6.1.12
        jupyter-console==6.4.0
        jupyter-core==4.7.1
        jupyter-server==1.10.1
        notebook==6.4.0
        numpy==1.20.3
        pandas==1.2.4
        scikit-learn==0.24.2
        scipy==1.6.3
        torch==1.8.1
        torchvision==0.9.1
        voila==0.2.10
        kaggle
        pymongo
    
    pip install -r requirements.txt
    
Installing these python libraries will give an incompatibility with fastai requiring nbconvert<6 and voila requiring nbconvert>=6. Although not ideal, it seems we can ignore this for now.

### Copy model to server from GPU computer

    scp learn_empty_dls_0708.pkl ubuntu@tabulous.pythonator.com:jupyter/homesite-quote/

### Set up kaggle api

    mkdir ~/.kaggle
    nano ~/.kaggle/kaggle.json
        {"username":"yourusername","key":"yourkey"}

### Install app in jupyter folder

    cd ~/jupyter
    git clone https://github.com/timcu/fast-tabulous-app.git

### Configure Voila as a service and start

    sudo nano /etc/systemd/system/voila.service
        [Unit]
        Description=Voila
        [Service]
        Type=simple
        PIDFile=/run/voila.pid
        ExecStart=/home/ubuntu/jupyter/venv/bin/voila --no-browser --show_tracebacks=True fast-tabulous-app/fast-tabulous-with-db.ipynb
        User=ubuntu
        WorkingDirectory=/home/ubuntu/jupyter
        Restart=always
        RestartSec=10
        [Install]
        WantedBy=multi-user.target
        
    sudo systemctl daemon-reload
    sudo systemctl start voila  # start now
    sudo systemctl enable voila  # start after every reboot or failure

## Configure nginx

    sudo nano /etc/nginx/sites-available/tabulous.pythonator.com
        server {
            server_name tabulous.pythonator.com;
            root /var/www/tabulous.pythonator.com;
            proxy_buffering off;
            location / {
                    proxy_pass http://localhost:8866;
                    proxy_set_header Host $host;
                    proxy_set_header X-Real-IP $remote_addr;
                    proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;

                    proxy_http_version 1.1;
                    proxy_set_header Upgrade $http_upgrade;
                    proxy_set_header Connection "upgrade";
                    proxy_read_timeout 86400;
            }
            client_max_body_size 100M;
            error_log /var/log/nginx/error.log;
            listen 80;
            listen [::]:80;
        }

    cd /etc/nginx/sites-enabled
    sudo ln -s ../sites-available/tabulous.pythonator.com
    
Now create a certificate for the site choosing redirect when asked the question

    sudo certbot --nginx -d tabulous.pythonator.com  # 2 - redirect

It may be necessary to move all the proxy lines from 80 to 443 after reconfigured

## Visit the site and check it all works

[https://tabulous.pythonator.com](https://tabulous.pythonator.com)

## Troubleshooting jupyter

Connect to server tunnelling port 8888

    ssh -L 8888:localhost:8888 ubuntu@tabulous.pythonator.com
    cd jupyter
    source venv/bin/activate
    jupyter notebook

Then enter URL into browser on local computer. Running notebook should show log messages in console.

## Troubleshooting voila

Connect to server tunnelling port 8866

    ssh -L 8866:localhost:8866 ubuntu@tabulous.pythonator.com
    cd jupyter
    source venv/bin/activate
    voila

Then enter URL into browser on local computer. Running notebook should show log messages in console.

## Trougbleshooting voila service

    sudo systemctl status voila

## Troubleshooting nginx

Check logs in /var/log/nginx